### AsyncIO

- 비동기 I/O Coroutine 작업
- Generator → 반복적인 객체 Return 사용
- Non-blocking 비동기 처리

- asytncio(Asynchronous I/O)는 비동기 프로그래밍을 위한 모듈이며 CPU작업과 I/O를 병렬로 처리하게 해줌

동기(synchronous) 처리는 특정 작업이 끝나면 다음 작업을 처리하는 순차처리 방식이고,<br>
비동기(asynchronous) 처리는 여러 작업을 처리하도록 예약한 뒤 작업이 끝나면 결과를 받는 방식임

- 네이티브 코루틴 만들기
Python에서는 generator 기반의 코루틴과 구분하기 위해 async def로 만든 코루틴은 네이티브 코루틴이라고 함.

In [6]:
import asyncio

async def hello(): # async def로 네이티브 코루틴을 만듦
    print('Hello, world!')
    
loop = asyncio.get_event_loop()  # 이벤트 루프를 얻음
loop.run_until_complete(hello()) # hello가 끝날 때까지 기다림
loop.close()                     # 이벤트 루프를 닫음

- 이벤트루프 = asyncio.get_event_loop()
- 이벤트루프.run_until_complete(코루틴 객체 또는 퓨처 객체)

run_until_complete는 네이티브 코루틴이 이벤트 루프에서 실행되도록 예약하고, 해당 네이티브 코루틴이 끝날때까지 기다림<br>
이벤트 루프를 통해서 hello 코루틴이 실행됨 <br>
할 일이 끝났으면 loop.close로 이벤트 루프를 닫아줌<br>


- await로 네이티브 코루틴 실행하기

다음과 같이 await 뒤에 코루틴 객체, 퓨처 객체, 테스크 객체를 지정하면 해당 객체가 끝날 때까지 기다린 뒤 결과를 반환함.<br>
await 단어 뜻 그대로 특정 객체가 끝날 때까지 기다림<br>
await 키워드는 python 3.5이상부터 사용가능, 3.4에서는 yield from을 사용

- 변수 = await 코루틴 객체
- 변수 = await 퓨처 객체
- 변수 = await 테스크 객체

여기서 주의할 점이 있는데 await는 네이티브 코루틴 안에서만 사용할 수 있음

- Blocking I/O: 호출된 함수가 자신의 작업이 완료될 때까지 제어권을 가지고 있음. 타 함수는 대기
- NonBlocking I/O: 호출된 함수가(서브루틴) return 후 호출한 함수(메인루틴)에 제어권 전달 → 타 함수는 일 지속

Thread 단점: 디버깅, 자원 접근 시 레이스 컨디션(경쟁상태), 데드락(Dead Lock) → 고려 후 코딩 <br>
Coroutine 장점: 하나의 루틴만 실행 → 락 관리 필요X → 제어권으로 실행<br>
Coroutine 단점: 사용함수가 비동기로 구현이 되어 있어야 하거나, 또는 직접 비동기로 구현해야한다.

AsyncIO 실습<br>
1) 가상환경 생성<br>
2) 타겟 사이트 살펴보기<br>
3) 프로그램 설명<br>
4) async, await 설명<br>
5) 프로그램 테스트<br>

### Asyncio 웹 스크래핑

In [ ]:
import asyncio
import timeit
from urllib.request import urlopen
from concurrent.futures import ThreadPoolExecutor
import threading

# 실행 시작 시간
start = timeit.default_timer()
urls = ['http://daum.net', 'https://naver.com', 'https://tistory.com']

async def fetch(url,executor):
    # 실행
    res = await loop.run_in_executor(executor, urlopen, url)
    
    # 결과 반환
    return res.read()


async def main():
    
    # 스레드 풀 생성
    executor = ThreadPoolExecutor(max_workers=10)
    
    # future 객체 모아서 gather에서 실행
    futures =[
        asyncio.ensure_future(fetch(url,executor)) for url in urls
    ]
    
    # 결과 취합
    rst = await asyncio.gather(*futures)
    
    print()
    print('Result:',rst)
    
if __name__ == "__main__":
    # 루프 초기화
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    duration = timeit.default_timer() - start
    
    print('Total Running Time:',duration)

### beautifulsoup을 이용한 것

In [ ]:
import asyncio
import timeit
from urllib.request import urlopen
from concurrent.futures import ThreadPoolExecutor
import threading
from bs4 import BeautifulSoup

import sys
import io
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding = 'utf-8')
sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding = 'utf-8')

# 실행 시작 시간
start = timeit.default_timer()
# 서비스 방향이 비슷한 사이트로 실습 권장(예 : 게시판성 커뮤니티)
urls = ['http://daum.net', 'https://naver.com', 'http://mlbpark.donga.com/', 'https://tistory.com', 'https://www.inflearn.com/']


async def fetch(url, executor):
    # 쓰레드명 출력
    print('Thread Name :', threading.current_thread().getName(), 'Start', url)
    # 실행
    res = await loop.run_in_executor(executor, urlopen, url)
    
    soup = BeautifulSoup(res.read(), 'html.parser')
    
    # 전체 페이지 소스 확인
    # print(soup.prettify())
    # 이 부분에서 BeautifulSoup Selector(선택자)를 활용해서 다양한 정보 가져오기 가능
    # 현 예제에서는 페이지 타이틀 정보 수집
    result_data = soup.title

    print('Thread Name :', threading.current_thread().getName(), 'Done', url)
    # 결과 반환
    return result_data

async def main():
    # 쓰레드 풀 생성
    executor = ThreadPoolExecutor(max_workers=10)

    # future 객체 모아서 gather에서 실행
    futures = [
        asyncio.ensure_future(fetch(url, executor)) for url in urls
    ]
    
    # 결과 취합
    rst = await asyncio.gather(*futures)

    print()
    print('Result : ', rst)

if __name__ == '__main__':
    # 루프 초기화
    loop = asyncio.get_event_loop()
    # 작업 완료 까지 대기
    loop.run_until_complete(main())
    # 수행 시간 계산
    duration = timeit.default_timer() - start
    # 총 실행 시간
    print('Total Running Time : ', duration)